[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PicselliaTeam/picsellia-training-engine/blob/master/Picsellia-Yolov5-Training.ipynb)

In [1]:
!pip install picsellia_yolov5
!pip install picsellia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 5.5 MB/s eta 0:00:00
  Using cached pandas-1.1.5-cp38-cp38-manylinux1_x86_64.whl (9.3 MB)
  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
  Using cached Cython-0.29.21-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Using cached scipy-1.4.1-cp38-cp38-manylinux1_x86_64.whl (26.0 MB)
  Using cached Pillow-7.2.0-cp38-cp38-manylinux1_x86_64.whl (2.2 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 2.2 MB/s eta 0:00:0000:0100:01
  Using cached matplotlib-3.3.4-cp38-cp38-manylinux1_x86_64.whl (11.6 MB)
  Using cached opencv_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.4 MB)
  Using cached pycocotools-2.0.2-cp38-cp38-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

🥑 **Welcome To Picsellia Training Quickstart Notebook for YOLOv5** 🥑

In this Notebook, you will see how to launch a YOLOv5 training from a created experiment on the Platform and log all the evaluation metrics to analyse your trained model.

**Step 1, let's import our python SDK and our tensorflow2 wrapper** 

If you do not have our packages you can run: 
- `pip install picsellia picsellia_yolov5` 


> This notebook is an `easy mode` for YOLOv5 Training, you don't have to worry about nothing

In [1]:
from picsellia import Client
from picsellia_yolov5.utils import to_yolo, find_matching_annotations, edit_model_yaml, generate_yaml, Opt, setup_hyp
from picsellia_yolov5.utils import send_run_to_picsellia
from picsellia_yolov5.yolov5.train import train

Error while loading conf file for logging. No logging done.


**Step 2, fetch your experiment parameters and files from the Picsellia servers**

Every experiment you make has an unique identifier allowing you to retrieve all the necessary informations with one command.

By passing `tree` and `with_files` to True, you will automatically get a folder architecture like:

- `experiment_name`/
    - checkpoint/
    - config/
    - exported_model/
    - images/
    - metrics/
    - records/
    - results/

In [ ]:
api_token = ''
project_name = ''
experiment_name = ''

client = Client(
    api_token=api_token,
    organization=None # Set to an organization name if you want to checkout an other org.
)

project= client.get_project(project_name)

experiment = project.get_experiment(
    name=experiment_name, 
    tree=True, 
    with_file=True
)

**Step 3, Dataset fetching** 

Download all the necessary data for your training:
- Dataset
- Annotations

Then generate the labelmap for your model, `labelmap` is needed for Picsellia to map your verbose labels *(i.e "cat", "dog", "hot-dog")* to your categorical labels *(i.e 1, 2, 3)*.

>
> You can find more info about the labelmap format [here](https://google.com)
>

Finally performing train-test-split to perform training, `train_test_split`  is recommended to be stored inside Picsellia, this way you will be able to access the validation interface and have full visibility over your training data 👊

If you want to know more about our train_test_split format, here is the [documentation page](https://google.com)
**(default repartition is 0.8 / 0.2 from train/test)**

Once the train test set created, we send the repartition to Picsellia platform in order to visualize it later.


In [ ]:
experiment.download_annotations()
experiment.download_pictures()
experiment.generate_labelmap()
experiment.log('labelmap', experiment.label_map, 'labelmap', replace=True)

YOLODIR = 'YOLO-{}'.format(experiment_name)
train_set, test_set = to_yolo(pxl_annotations_dict=experiment.dict_annotations,labelmap=experiment.label_map, base_imgdir=experiment.png_dir, targetdir=YOLODIR, prop=0.7, copy_image=False) 

train_split = {
    'x': train_set["categories"],
    'y': train_set["train_repartition"],
    'image_list': train_set["image_list"],
}
experiment.log('train-split', train_split, 'bar', replace=True)

test_split = {
    'x': test_set["categories"],
    'y': test_set["train_repartition"],
    'image_list': test_set["image_list"],
}
experiment.log('test-split', test_split, 'bar', replace=True)

**Step 4, Pre-processing**

We provide wrapper functions to generate the config `.yaml` file needed by YOLOv5 github repo.



In [ ]:
generate_yaml(
    yamlname=experiment_name,
    targetdir=YOLODIR,
    labelmap=experiment.label_map
)
cfg = edit_model_yaml(
    label_map=experiment.label_map,
    experiment_name=experiment_name,
    config_path=experiment.config_dir

)
hyp, opt, device = setup_hyp(
    experiment_name, 
    cfg,
    experiment.checkpoint_dir,
    experiment.get_log('parameters'),
    experiment.label_map
)

**Step 5 - Launch training 🚀**

In [ ]:
train(hyp, opt, opt.device, pxl=experiment)

**Step 6 - Store your trained Model in Picsellia to use it later 🥑**

In [ ]:
send_run_to_picsellia(experiment, YOLODIR)